PR-new-logo-orizontal (1).svg
# **Lab03PR. Analiza univariata si bivariata a datelor (2 ore pentru rezolvare)**



Copyrigth (c) 2022 Teodor-Andrei Neacsu teodor.neacsu@upb.ro

#### Overview:
+ Analiza univariată + vizualizări
    + distributția normala
    + Shapiro test
+ Analiză bivariată + vizualizări
+ Corelația Pearson
+ Standardizare și normalizare
+ PCA

În acest laborator vom parcurge metode de a obține intuiții despre setul/seturile de date cu care lucrăm.

#### De ce sunt importante analizele univariate și bivariate?
Folosind analiza univariată:
+ identificăm distribuția datelor
+ identificăm valori extreme (outliers)
+ pentru sanitizarea/curățarea datelor

Folosind analiza bivariată și corelațiile dintre variabile:
+ înțelegem tendințele si trendurile din date
+ identificăm relații între variabile


### Dataset

În acest laborator vom folosi datasetul [Kaggle Vehicle Dataset](https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho?ref=hackernoon.com&select=Car+details+v3.csv). În folderul `data` se află `vehicle_dataset.csv` care conține informații despre vehiculele utilizate.

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Incarcam datele intr-un dataframe
car_df = pd.read_csv('./data/vehicle_dataset.csv')
car_df.head()

## Analiza univariată


Este cea mai simplă formă de analiză a datelor în care datele analizate conțin o ```singură variabilă```. Deoarece este efectuată pe o singură variabilă, nu se ocupă de cauze sau relații. Scopul principal al analizei univariate este de a descrie datele și de a găsi caracteristicile principale ale acestora. Pana la a găsi caracteristicile datelor, vom face o clasificare a acestora in functie de tipul de date pe care îl conțin.

### Tipuri de date

+ **Date numerice** - sunt date care pot fi măsurate. Ex: temperatura, greutatea, înălțimea, etc.
+ **Date categorice** - sunt date care nu pot fi măsurate și au fost grupate in seturi de categorii mutual exclusive. Ex: culoarea, tipul, etc.
    + **Date ordinale / ordonate (ordonabile)** - sunt date categorice care pot fi ordonate. Ex: mărimea, gradul de satisfacție, etc.
    + **Date nominale / neordonate (neordonabile)** - sunt date categorice care nu pot fi ordonate. Ex: culoarea, tipul, etc.

<img src="./resources/data_clf.png" alt="" width="800"/>

In [ ]:
# pentru a observa tipul de date din fiecare coloana folosim functia info()
car_df.info()

### Ce putem folosi pentru a descrie o variabilă pentru un set de date?

Pentru **datele numerice** putem folosi urmatoarele statistici:
- **media** - suma tuturor valorilor împarțită la numărul de valori.
- **mediana** - valoarea din mijloc (din distribuție) a datelor ordonate crescător.
- **mode** - valoarea care apare cel mai des (in distribuția datelor) în setul de date.
- **deviația standard** - o măsură a cantității de variație sau dispersie a unui set de valori. O deviație standard scăzută indică faptul că valorile tind să fie apropiate de media (numită și valoare așteptată) a setului, în timp ce o abatere standard mare indică faptul că valorile sunt răspândite într-un interval mai larg.

Deviația standard are următoarea formulă:

$$\sigma = \sqrt{\frac{\sum_{i=1}^{n}(x_i - \mu)^2}{n}}$$

unde $\mu$ este media datelor.

Pentru **datele categorice** putem folosi următoarele statistici:
- **mode** - valoarea care apare cel mai des in setul de date.
- **frecvența absolută** - numărul de apariții al unei valori
- **frecvența relativă** - procentul de apariții al unei valori

In [ ]:
# pentru datele numerice putem folosi functia describe()
car_df.describe()

In [ ]:
# pentru datele categorice putem folosi funcția describe(include=['O'])
car_df.describe(include=['O'])

In [ ]:
# pentru a afișa frecvența absolută a valorilor dintr-o coloană folosim funcția value_counts()
car_df["name"].value_counts()

In [ ]:
# pentru frecvența relativă folosim funcția value_counts(normalize=True)
car_df["name"].value_counts(normalize=True)

## Distribuția normală

Distribuția normală este o distribuție de probabilitate continuă, care este caracterizată prin faptul ca are formă de clopot. Este o distributie simetrică, astfel ca media, mediana si moda sunt egale. Distribuția normală este folosita in multe domenii, deoarece este o distribuție care se apropie de realitatea datelor. 

Formula distribuției normale este:

$$f(x) = \frac{1}{\sigma \sqrt{2\pi}} e^{-\frac{(x-\mu)^2}{2\sigma^2}}$$

unde:
- $\mu$ este media
- $\sigma$ este deviația standard

În figura de mai jos putem observa:
+ În mijloc distribuția normala
+ În stânga se află distribuția cu skew negativ
+ În dreapta distribuția cu skew pozitiv.

Skew este o masură a simetriei unei distribuții. O distribuție simetrică are skew = 0, iar o distribuție asimetrică are skew < 0 (skew negativ) sau skew > 0 (skew pozitiv). Formula este dată mai jos

$$SKEW = \frac{3 * (Mean - Median)}{Standard Deviation}$$

Skew pentru:
+ distribuție simetrica este 0
+ distribuție cu skew negativ este < 0
+ distribuție cu skew pozitiv este > 0


<img src="./resources/distr.png" alt="" width="800"/>

In [ ]:
# pentru a calcula skewness-ul folosim funcția skew()
car_df.skew()

## De ce ne interesează distribuția normală? 

Modelele de ML au o performanță mai bună atunci când datele sunt distribuite normal. Trebuie să vedem ce fel de date avem in setul nostru de date si sa testăm dacă sunt distribuite normal.

Trebuie să verificăm daca datele sun distribuite normal

+ prin vizualizare
+ prin testul de **Shapiro-Wilk**


### Ipoteza nulă (H0) si ipoteza alternativă (H1) pentru testul de Shapiro-Wilk

Ipoteza nulă (H0) este ipoteza care presupune că datele sunt distribuite normal.

Ipoteza alternativă (H1) presupune că datele NU sunt distribuite normal.


### Shapiro-Wilk test

Dacă valoarea p este mai mică decât nivelul alpha ales, atunci ipoteza nulă este respinsă și există dovezi că datele testate nu sunt distribuite normal. 

Pe de altă parte, dacă valoarea p este mai mare decât nivelul alpha, atunci ipoteza nulă (că datele provin dintr-o populație distribuită normal) nu poate fi respinsă (de exemplu, pentru un nivel alfa de .05, un set de date cu o valoare p mai mică de .05 respinge ipoteza nulă conform căreia datele provin dintr-o populație distribuită normal). 

În consecință, un set de date cu o valoare p mai mare decât valoarea alpha .05 nu reușește să respingă ipoteza nulă că datele provin dintr-o distribuție normală).


In [ ]:
# shapiro test
# H0: distribuția este normală
# H1: distribuția nu este normală
# alpha = 0.05
# dacă p-value < alpha => respingem H0 => distribuția nu este normală
# dacă p-value > alpha => nu respingem H0 => distribuția este normală

# pentru a verifica daca distribuția este normală folosim funcția shapiro()
# funcția returnează statistic si p-value
# p-value = p
_, p = stats.shapiro(car_df["selling_price"][:500])

print("p-value = ", p)

## Tipuri de grafice / vizualizări pentu analiză univariată

## Histograma

Histogramele reprezintă o metodă de vizualizare a distribuției numerice a datelor. Parametrul principal al acestui grafic este numărul de bin-uri folosite. Aceste **bin-uri** se referă la numărul de bar-uri care vor reprezenta distribuția.

Numărul de bin-uri ales decide dacă vizualizarea pe care o facem este useful sau useless. Nu există o metoda bine definită prin care să decidem ce număr de bin-uri folosim. Dacă numărul de bin-uri este prea mic, informația se va pierde. Dacă numărul de bin-uri este prea mare, putem ajunge la situația în care fiecare sample va fi încadrat într-un bin-ul său propriu, iar vizualizarea va deveni inutilă.


In [ ]:
plt.figure(figsize=(10, 8))
sns.histplot(x='selling_price', data=car_df, bins=70)
plt.show()

### KDE (Kernel Density Estimation Plot) - Estimarea densității de probabilitate

KDE reprezintă o variantă de histogramă care este mai smooth (mai neteda) si mai bună pentru a vedea distribuția datelor.

In [ ]:
plt.figure(figsize=(10, 8))
sns.kdeplot(x='km_driven', data=car_df)
plt.show()

## Boxplot

Boxplot-urile reprezintă o metodă de vizualizare a datelor ce include 5 valori statistice:

+ Minimum
+ Prima quartilă
+ Mediana (A doua quartilă)
+ A treia quartilă
+ Maximum

**Quartilă (Quartile) = 25% din dataset**

Acest tip de vizualizare împarte datele în secțiuni de aproximativ 25%. Valorile extreme (outliers) sunt reprezentate prin puncte.

<img src="./resources/box.png" alt="" width="600"/>

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(x='year', data=car_df)
plt.show()

## Violin plot (KDE + Boxplot)

Este o combinație între KDE si Boxplot. Include cele 5 valori statistice si arată distribuția datelor.

In celula următoare aveți o descriere a acestui tip de grafic.

<img src="./resources/violin.png" alt="" width="600"/>

In [ ]:
plt.figure(figsize=(10, 5))
sns.violinplot(x='year', data=car_df)
plt.show()

## Analiză bivariată - Corelații

Analiza bivariată este folosită pentru studiul relației dintre două variabile. Analiza bivariată ajută la testarea ipotezei cauzalității și asocierii. Ajută la prezicerea valorii unei variabile care este dependentă de modificările unei alte variabile independente.

## Tipuri de grafice / vizualizări pentu analiza bivariată

### Scatter plot

Scopul Scatter Plot-urilor este de a vizualiza relatia dintre 2 variabile folosind puncte ca metoda de reprezentare. In urmatorul exemplu pe axa Ox avem numarul de kilometrii parcursi iar pe axa Oy avem pretul masinii.

In [ ]:
plt.figure(figsize=(10, 5))
sns.scatterplot(x='km_driven', y='selling_price', data=car_df)
plt.xlim(0, 1000000)
plt.ylim(0, 10000000)
plt.show()

### Lineplot

In [ ]:
plt.figure(figsize=(10, 5))
sns.lineplot(x='year', y='selling_price', data=car_df)
plt.show()

### Regplot

Regplot este un scatter plot cu o linie de regresie (linie de tendință).

In [ ]:
plt.figure(figsize=(10, 5))
sns.regplot(x='year', y='km_driven', data=car_df)
plt.show()

#### Corelația Pearson

Corelația Pearson r este cea mai utilizată statistică de corelație pentru a măsura gradul de relație dintre variabilele înrudite liniar. De exemplu, în piața de valori, dacă dorim să măsurăm modul în care două acțiuni sunt legate între ele, corelația Pearson r este utilizată pentru a măsura gradul de relație dintre cele două.

Presupuneri:
+ cele 2 variabile sunt normal distribuite
+ cele 2 variabile liniar corelate


<img src="./resources/r_.png" alt="Pearson_correlation_coefficient" width="600"/>

![](resources/r.png)

In [ ]:
# calculăm corelația dintre variabilele numerice folosind funcția corr()
car_df.corr(method='pearson')

### Cauzalitatea corelațiilor

În cazul în care gasim o corelație între 2 variabile A și B există 5 posibilități:
+ A poate fi cauza lui B
+ B poate fi cauza lui A
+ A și B sunt corelate dar sunt cauzate de o altă variabilă C
+ poate exista o cauzalitate in lanț: A cauzează E, E cauzează B (A -> E -> B)

Câteva exemple:
+ Corelație negativă între păr și venit.
+ Corelație pozitivă între numărul de pompieri de la un incendiu și daunele materiale.
+ Corelație pozitivă între cantitatea de alcool și numărul de manele ascultate la un party.

#### !!! În cazul în care avem corelații puternice între variabilele din setul de date, vom elimina toate variabilele care sunt corelate exceptând una dintre ele deoarece variabilele corelate sunt redundante pentru modelele de ML.

Acest proces face parte din task-ul de **selectare a feature-urilor(feature selection)**.

### Heatmap-ul corelațiilor

În continuare vom vedea cum putem vizualiza corelațiile dintre variabilele numerice dintr-un dataset folosind heatmap.

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(car_df.corr(method='pearson'), annot=True, cmap='coolwarm')
plt.show()

### Pairplot

După calculul coeficienților de corelație, putem selecta variabilele de interes si să le vizualizăm folosind pairplot.

In [ ]:
plt.figure(figsize=(10, 8))
sns.pairplot(car_df[['selling_price', 'km_driven', 'year']])
plt.show()

### Standardizare si Normalizare

Normalizarea este o tehnică de scalare în care valorile sunt deplasate și redimensionate astfel încât acestea ajung să varieze între 0 și 1. Este cunoscută și sub numele de scalare Min-Max.

$$x_{norm} = \frac{x - min(x)}{max(x) - min(x)}$$

Standardizarea este o tehnică de scalare în care valorile sunt centrate în jurul mediei cu o abatere standard unitară. Aceasta înseamnă că media atributului devine zero și distribuția rezultată are o abatere standard unitară.

$$x_{std} = \frac{x - mean(x)}{std(x)}$$

Rule of thumb:
+ Dacă datele sunt distribuite normal, atunci standardizarea este preferată
+ Dacă datele nu sunt distribuite normal, atunci normalizarea este preferată

De ce sunt utile aceste preprocesări ale datelor ?
+ Când veți antrena modele si veți folosi tehnici de optimizare precum gradient descent veți vedea că dacă feature-urile au range-uri foarte diferite, dimensiunea pasului de optimizare va varia in functie de range-urile feature-urilor. Dorim ca dimensiunea gradienților să fie similară pentru toate feature-urile pentru a ajunge rapid la parametrii optimi.  


<img src="./resources/grad_desc.png" alt="Scaling when using gradient descent" width="900"/>

In [ ]:
### Normalizarea datelor folosind MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

# creăm un obiect de tip MinMaxScaler
scaler = MinMaxScaler()

# aplicăm funcția fit_transform() pe coloana selling_price
normed = scaler.fit_transform(car_df[['selling_price']])

### Standardizarea datelor folosind StandardScaler
from sklearn.preprocessing import StandardScaler

# creăm un obiect de tip StandardScaler
scaler = StandardScaler()

# aplicăm functia fit_transform() pe coloana selling_price
scaled = scaler.fit_transform(car_df[['selling_price']])


# Inainte si după
plt.figure(figsize=(10, 8))
sns.kdeplot(x=normed.flatten())
sns.kdeplot(x=scaled.flatten())
plt.show()



Am văzut cum putem vizualiza corelații folosind 2 variabile. Dar cum putem vizualiza corelațiile dintre mai multe variabile?

### PCA (Principal Component Analysis)

PCA este o metodă de reducere a dimensiunii a datelor. Aceasta transformă un set de date cu n dimensiuni intr-un set de date cu k dimensiuni (k < n) prin proiectarea datelor pe un subspațiu de dimensiune k. In acest laborator vom folosi PCA pentru a vizualiza datele <=> vom reduce dimensiunea datelor de la n la k = 2.

PCA este util pentru următoarele task-uri:
+ analizarea dataseturilor cu multe feature-uri
+ reducerea dimensiunii datelor
+ vizualizarea datelor
+ clustering

\* Și pentru PCA pașii de preprocesare a datelor (standardizare si normalizare) sunt benefici.

PCA presupune crearea unei ierarhii de componente principale (PC) care descriu datele (se bazează pe SVD). Componentele principale sunt axele principale ale datelor. Axele sunt ordonate în funcție de variația datelor pe acestea. Prima componentă principala are cea mai mare variație, a doua componentă principala are cea mai mare variație din cea ramasă dupa ce s-a scos variația de pe prima componentă, etc.



<img src="./resources/pca.jpg" alt="Pearson_correlation_coefficient" width="900"/>

In [ ]:
# exemplu de PCA pe datele din car_df
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

car_df.dropna(inplace=True)

# scale car_df
scaler = StandardScaler()
scaled_car_df = scaler.fit_transform(car_df[['selling_price', 'km_driven', 'year', 'seats']])

# aplicăm PCA pe datele scalate
pca = PCA(n_components=2)
pca.fit(scaled_car_df)

# transformăm datele scalate in date proiectate pe componentele principale
pca_car_df = pca.transform(scaled_car_df)

# vizualizăm datele proiectate pe componentele principale
plt.figure(figsize=(10, 8))
sns.scatterplot(x=pca_car_df[:, 0], y=pca_car_df[:, 1], hue=car_df['fuel'])
plt.title('PCA')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.xlim(-5, 5)
plt.ylim(-5, 8)
plt.show()



## Exerciții

Pentru exerciții vom folosi datasetul [Loan Predication](https://www.kaggle.com/datasets/ninzaami/loan-predication).

In [ ]:
loan_df = pd.read_csv('./data/loan.csv')
loan_df.head()

In [ ]:
loan_df.describe()

#### 1. Efectuați analiză univariată pe dataset. Folosiți cele 3 tipuri de grafice. Pentru graficul cu histograma afișați media si mediana (hint: plt. axvline).  

In [ ]:
plt.figure(figsize=(10, 8))
sns.kdeplot(x='LoanAmount', data=loan_df)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(x='LoanAmount', data=loan_df)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.violinplot(x='LoanAmount', data=loan_df)
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.histplot(x='LoanAmount', data=loan_df, bins=70)
plt.axvline(x=loan_df['LoanAmount'].mean())
plt.axvline(x=loan_df['LoanAmount'].median())
plt.show()

#### 2. Efectuați analiză bivariată pe dataset. Folosiți cele puțin 3 tipuri de grafice. Ce observații puteți face?

In [ ]:
plt.figure(figsize=(10, 5))
sns.scatterplot(x='LoanAmount', y='Loan_Amount_Term', data=loan_df)
plt.show()

In [ ]:
sns.lineplot(x='LoanAmount', y='Loan_Amount_Term', data=loan_df)
plt.show()

In [ ]:
sns.regplot(x='LoanAmount', y='Loan_Amount_Term', data=loan_df)
plt.show()

#### 3. Plotați heatmap-ul corelațiilor apoi pairplot. Ce observații puteti face?

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(loan_df.corr(method='pearson'), annot=True, cmap='coolwarm')
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.pairplot(loan_df[['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']])
plt.show()

#### 4. Calculați coeficientul de corelație Pearson intre 2 coloane din dataset. Folosiți formula de mai sus si operațiile pe coloane din pandas. **NU folosiți functia de corelație din pandas.**

In [ ]:
loan_df.dropna(inplace=True)


my_covariance = loan_df[["ApplicantIncome", "LoanAmount"]].cov()
print(my_covariance)
PearsonCo = my_covariance.loc['LoanAmount'] / (loan_df["ApplicantIncome"].std() * loan_df["LoanAmount"].std())

print("PearsonCo")
PearsonCo

#### 5. Efectuați PCA pe dataset-ul standardizat. Plotați graficul cu componentele principale. Ce observații puteți face?

In [ ]:
loan_df.dropna(inplace=True)

# scale car_df
scaler = StandardScaler()
scaled_loan_df = scaler.fit_transform(loan_df[['CoapplicantIncome', 'LoanAmount', 'Credit_History']])

# aplicăm PCA pe datele scalate
pca = PCA(n_components=2)
pca.fit(scaled_loan_df)

# transformăm datele scalate in date proiectate pe componentele principale
pca_loan_df = pca.transform(scaled_loan_df)

# vizualizăm datele proiectate pe componentele principale
plt.figure(figsize=(10, 8))
sns.scatterplot(x=pca_loan_df[:, 0], y=pca_loan_df[:, 1], hue=loan_df['ApplicantIncome'])
plt.title('PCA')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()